### Run in Google CoLab! (Open in new window or new tab)
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/m-wessler/nbm-verify/blob/master/notebooks/verify_1Dqpf_dev.ipynb)

In [1]:
import os
import sys
import csv
import nbm_funcs
import requests

import numpy as np
import pandas as pd
import xarray as xr

import seaborn as sns
import scipy.stats as scipy
import urllib.request as req
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

import warnings
warnings.filterwarnings('ignore')

***
***
# Configuration
Select 'site' to evaluate, modify 'vsite' if an alternate verification site is preferred<br>
Fixed 'date0' at the start of the NBM v3.2 period (2/20/2020)<br>
Full lead time is 263 hours - Note if date1 is within this period, there will be missing verification data as it does not exist yet!

In [2]:
# NBM 1D Viewer Site to use
site = nbm_funcs._site = 'KMSO'
vsite = site

# Data Range
lead_time_end = 263
init_hours = [13]#[1, 7, 13, 19]

date0 = nbm_funcs._date0 = datetime(2020, 3, 1)
date1 = nbm_funcs._date1 = datetime(2020, 7, 21)

In [3]:
sitepath = site if site == vsite else '_'.join([site, vsite])

datadir = nbm_funcs._datadir = '../archive/%s/data/'%sitepath
os.makedirs(datadir, exist_ok=True)

figdir = nbm_funcs._figdir = '../archive//%s/figures/'%sitepath
os.makedirs(figdir, exist_ok=True)

dates = pd.date_range(date0, date1, freq='1D')
date2 = nbm_funcs._date2 = date1 + timedelta(hours=lead_time_end)

print(('\nForecast Site: {}\nVerif Site: {}\nInit Hours: '+
      '{}\nFirst Init: {}\nLast Init: {}\nLast Verif: {}').format(
    site, vsite, init_hours, date0, date1, date2))


Forecast Site: KMSO
Verif Site: KMSO
Init Hours: [13]
First Init: 2020-03-01 00:00:00
Last Init: 2020-07-21 00:00:00
Last Verif: 2020-07-31 23:00:00


***
***
# Obtain observation data from SynopticLabs (MesoWest) API
These are quality-controlled precipitation observations with adjustable accumulation periods<br>
See more at: https://developers.synopticdata.com/mesonet/v2/stations/precipitation/
<br><br>
If no observation file exists, will download and save for future use

In [4]:
# Get metadata for the select point
meta_base = 'https://api.synopticdata.com/v2/stations/metadata?'
api_token = '&token=a2386b75ecbc4c2784db1270695dde73'
meta_site = '&stid=%s&complete=1'%site
url = meta_base + api_token + meta_site
# print(url)

site_meta_raw = requests.get(url).json()
# print(meta_raw['STATION'][0])

zone = site_meta_raw['STATION'][0]['NWSZONE']
cwa = site_meta_raw['STATION'][0]['CWA']

print('Site: %s\nCWA: %s\nZone: %s'%(site, cwa, zone))

Site: KMSO
CWA: MSO
Zone: MT005


In [5]:
# Get a list of sites in the CWA that report precip
precip_base = 'https://api.synopticdata.com/v2/stations/precip?&complete=1&interval=6'
zone_query = '&nwszone=%s'%zone
cwa_query = '&cwa=%s'%cwa
date_query = '&start=%s&end=%s'%(
    date0.strftime('%Y%m%d%H%M'),
    (date0+timedelta(hours=6)).strftime('%Y%m%d%H%M'))

# We could query for a list of relevant zones within a CWA here
# Then pass a list of zones to the zone query
# !Add later!

url = precip_base + api_token + zone_query + date_query
zone_meta_raw = requests.get(url).json()

meta = []
for station in zone_meta_raw['STATION']:
    meta.append({k:station[k] for k in station.keys() if type(station[k]) == str})
meta = pd.DataFrame(meta).set_index('STID')

meta

,NWSFIREZONE,ELEV_DEM,TIMEZONE,SGID,SHORTNAME,ELEVATION,GACC,STATUS,LONGITUDE,COUNTY,STATE,CWA,NWSZONE,ID,MNET_ID,NAME,COUNTRY,LATITUDE,WIMS_ID
STID,,,,,,,,,,,,,,,,,,,
KMSO,MSO108,3192.3,America/Denver,NR04,NWS/FAA,3199,NRCC,ACTIVE,-114.09250,Missoula,MT,MSO,MT005,277,1,"Missoula, Missoula International Airport",US,46.92083,NaN
COVM,MSO109,3589.2,America/Denver,NR06,AGRIMET,3597,NRCC,ACTIVE,-114.08333,Ravalli,MT,MSO,MT005,653,11,CORVALLIS,US,46.33333,NaN
SMTM8,MSO109,5731.6,America/Denver,NR06,RAWS,5650,NRCC,ACTIVE,-114.226822,Ravalli,MT,MSO,MT005,1758,2,SMITH CREEK,US,46.439581,242912
TS934,MSO106,3412.1,America/Denver,NR04,RAWS,3412,NRCC,ACTIVE,-114.100889,Missoula,MT,MSO,MT005,2862,2,BLUE MTN,US,46.820725,241513
STVM8,MSO109,3320.2,America/Denver,NR06,RAWS,3365,NRCC,ACTIVE,-114.090881,Ravalli,MT,MSO,MT005,2866,2,STEVI,US,46.513514,242904
LRCM8,MSO109,5554.5,America/Denver,NR06,RAWS,5507,NRCC,ACTIVE,-114.262708,Ravalli,MT,MSO,MT005,6477,2,LITTLE ROCK CREEK,US,46.037828,242914
C4884,MSO108,3415.4,America/Denver,NR04,APRSWXNET/CWOP,3399,NRCC,ACTIVE,-113.96154,Missoula,MT,MSO,MT005,14971,65,CW4884 Missoula,US,46.89423,NaN
DRBM8,MSO109,3881.2,America/Denver,NR06,HADS,3879,NRCC,ACTIVE,-114.17611,Ravalli,MT,MSO,MT005,20948,106,RANGER STATION AT DARBY NEAR MISSOULA 4SE,US,46.02778,NaN
E0591,MSO108,3366.1,America/Denver,NR06,APRSWXNET/CWOP,3383,NRCC,ACTIVE,-114.06517,Missoula,MT,MSO,MT005,34356,65,EW0591 Missoula,US,46.80833,NaN


In [6]:
obs = []

for stid in meta.index:
    
    obfile = datadir + '%s_obs_%s_%s.pd'%(stid, date0.strftime('%Y%m%d'), date1.strftime('%Y%m%d'))

    if os.path.isfile(obfile):
        # Load file
        iobs = pd.read_pickle(obfile)
        print('Loaded obs from file %s'%obfile)

    else:
        # Get and save file
        iobs = nbm_funcs.get_precip_obs(stid, date0, date2)
        iobs = iobs[0].merge(iobs[1], how='inner', on='ValidTime').merge(iobs[2], how='inner', on='ValidTime')
        iobs = iobs[[k for k in iobs.keys() if 'precip' in k]].sort_index()

        iobs.to_pickle(obfile)
        print('Saved obs to file %s\n'%obfile)
    
    iobs['Site'] = np.full(iobs.index.size, fill_value=stid, dtype='U10')
    iobs = iobs.reset_index().set_index(['ValidTime', 'Site'])
    obs.append(iobs)

obs = pd.concat(obs).sort_index()
    
mm_in = 1/25.4
obs *= mm_in
[obs.rename(columns={k:k.replace('mm', 'in')}, inplace=True) for k in obs.keys()]

# OPTIONAL! Drop NaN rows... may help elim lower qual data
# obs = obs.dropna()

obs[:25]

Loaded obs from file ../archive/KMSO/data/KMSO_obs_20200301_20200721.pd
Loaded obs from file ../archive/KMSO/data/COVM_obs_20200301_20200721.pd
Loaded obs from file ../archive/KMSO/data/SMTM8_obs_20200301_20200721.pd
Loaded obs from file ../archive/KMSO/data/TS934_obs_20200301_20200721.pd
Loaded obs from file ../archive/KMSO/data/STVM8_obs_20200301_20200721.pd
Loaded obs from file ../archive/KMSO/data/LRCM8_obs_20200301_20200721.pd
Loaded obs from file ../archive/KMSO/data/C4884_obs_20200301_20200721.pd
Loaded obs from file ../archive/KMSO/data/DRBM8_obs_20200301_20200721.pd
Loaded obs from file ../archive/KMSO/data/E0591_obs_20200301_20200721.pd
Loaded obs from file ../archive/KMSO/data/MTM04_obs_20200301_20200721.pd
Loaded obs from file ../archive/KMSO/data/F6178_obs_20200301_20200721.pd
Loaded obs from file ../archive/KMSO/data/MTM77_obs_20200301_20200721.pd


6h_precip_in  12h_precip_in  24h_precip_in
ValidTime           Site                                             
2020-03-02 00:00:00 E0591           0.0           0.00           0.00
                    KMSO            0.0           0.00           0.01
                    LRCM8           0.0           0.00           0.00
                    SMTM8           0.0           0.02           0.02
                    STVM8           0.0           0.00           0.00
                    TS934           0.0           0.00           0.00
2020-03-02 12:00:00 C4884           0.0           0.01            NaN
                    COVM            0.0           0.00            NaN
                    E0591           0.0           0.00            NaN
                    KMSO            0.0           0.00            NaN
                    LRCM8           0.0           0.00            NaN
                    MTM04           0.0           0.00            NaN
                    MTM77           0.0           0.00            NaN
                    SMTM8           0.0           0.00            NaN
                    STVM8           0.0           0.00            NaN
                    TS934           0.0           0.00            NaN
2020-03-02 18:00:00 COVM            0.0            NaN            NaN
                    DRBM8           0.0            NaN            NaN
                    E0591           0.0            NaN            NaN
                    F6178           0.0            NaN            NaN
                    KMSO            0.0            NaN            NaN
                    LRCM8           0.0            NaN            NaN
                    MTM04           0.0            NaN            NaN
                    MTM77           0.0            NaN            NaN
                    SMTM8           0.0            NaN            NaN

In [7]:
obs.describe().T

,count,mean,std,min,25%,50%,75%,max
6h_precip_in,6753.0,0.013786,0.063033,0.0,0.0,0.0,0.00,0.84
12h_precip_in,3335.0,0.026406,0.099575,0.0,0.0,0.0,0.00,1.28
24h_precip_in,3357.0,0.053161,0.176021,0.0,0.0,0.0,0.01,2.10


***
***
# Obtain NBM forecast data from NBM 1D Viewer (csv file API)
These are the NBM 1D output files extracted from the viewer with 3 set accumulation periods<br>
See more at: https://hwp-viz.gsd.esrl.noaa.gov/wave1d/?location=KSLC&col=2&hgt=1&obs=true&fontsize=1&selectedgroup=Default
<br><br>
If no forecast file exists, will download and save for future use. This can take some time.

In [ ]:
nbmfile = datadir + '%s_nbm_%s_%s.pd'%(site, date0.strftime('%Y%m%d'), date1.strftime('%Y%m%d'))

if os.path.isfile(nbmfile):
    # Load file
    nbm = pd.read_pickle(nbmfile)
    print('Loaded NBM from file %s'%nbmfile)

else:
    url_list = []
    for date in dates:
        for init_hour in init_hours:
            # For now pull from the csv generator
            # Best to get API access or store locally later
            base = 'https://hwp-viz.gsd.esrl.noaa.gov/wave1d/data/archive/'
            datestr = '{:04d}/{:02d}/{:02d}'.format(date.year, date.month, date.day)
            sitestr = '/NBM/{:02d}/{:s}.csv'.format(init_hour, site)
            url_list.append([date, init_hour, base + datestr + sitestr])

    # Try multiprocessing this for speed?
    nbm = np.array([nbm_funcs.get_1d_csv(url, this=i+1, total=len(url_list)) for i, url in enumerate(url_list)])
    nbm = np.array([line for line in nbm if line is not None])

    header = nbm[0, 0]
    
    # This drops days with incomplete collections. There may be some use
    # to keeping this data, can fix in the future if need be
    # May also want to make the 100 value flexible!
    nbm = np.array([np.array(line[1]) for line in nbm if len(line[1]) == 100])

    nbm = nbm.reshape(-1, nbm.shape[-1])
    nbm[np.where(nbm == '')] = np.nan

    # Aggregate to a clean dataframe
    nbm = pd.DataFrame(nbm, columns=header).set_index(
        ['InitTime', 'ValidTime']).sort_index()

    # Drop last column (misc metadata?)
    nbm = nbm.iloc[:, :-2].astype(float)
    header = nbm.columns

    # variables = np.unique([k.split('_')[0] for k in header])
    # levels = np.unique([k.split('_')[1] for k in header])

    init =  nbm.index.get_level_values(0)
    valid = nbm.index.get_level_values(1)

    # Note the 1h 'fudge factor' in the lead time here
    lead = pd.DataFrame(
        np.transpose([init, valid, ((valid - init).values/3600/1e9).astype(int)+1]), 
        columns=['InitTime', 'ValidTime', 'LeadTime']).set_index(['InitTime', 'ValidTime'])

    nbm.insert(0, 'LeadTime', lead)

    klist = np.array([k for k in np.unique([k for k in list(nbm.keys())]) if ('APCP' in k)&('1hr' not in k)])
    klist = klist[np.argsort(klist)]
    klist = np.append('LeadTime', klist)
    nbm = nbm.loc[:, klist]
    
    # Nix values where lead time shorter than acc interval
    for k in nbm.keys():
        if 'APCP24hr' in k:
            nbm[k][nbm['LeadTime'] < 24] = np.nan
        elif 'APCP12hr' in k:
            nbm[k][nbm['LeadTime'] < 12] = np.nan
        elif 'APCP6hr' in k:
            nbm[k][nbm['LeadTime'] < 6] = np.nan
        else:
            pass
    
    nbm.to_pickle(nbmfile)
    print('\nSaved NBM to file %s'%obfile)

# Convert mm to in
nbm = pd.DataFrame([nbm['LeadTime']] + [nbm[k] * mm_in for k in nbm.keys() if 'LeadTime' not in k]).T

# Display some basic stats
nbm.loc[:, ['APCP6hr_surface', 'APCP6hr_surface_70% level', 'APCP6hr_surface_50% level',
            'APCP12hr_surface', 'APCP12hr_surface_70% level', 'APCP12hr_surface_50% level',
            'APCP24hr_surface', 'APCP24hr_surface_70% level', 'APCP24hr_surface_50% level'
            ]].describe().T

#### Plot the distribution of precipitation observations vs forecasts for assessment of representativeness

In [ ]:
thresh_id = nbm_funcs._thresh_id = {'Small':[0, 1], 'Medium':[1, 2], 'Large':[2, 3], 'All':[0, 3]}

# 33rd, 67th percentile determined above
thresholds = nbm_funcs._thresholds = {interval:nbm_funcs.apcp_dist_plot(obs, nbm, interval) 
              for interval in [6, 12, 24]}

# Use fixed override if desired
# thresholds = {
#     6:[1, 2],
#     12:[1, 2],
#     24:[1, 2]}

thresholds

***
***
# Reorganize the data for analysis:
#### Isolate the forecasts by accumulation interval and lead time

In [ ]:
plist = np.arange(1, 100)

data = []
for interval in [6, 12, 24]:
    
    pkeys = np.array([k for k in nbm.keys() if '%dhr_'%interval in k])
    pkeys = np.array([k for k in pkeys if '%' in k])
    pkeys = pkeys[np.argsort([int(k.split('_')[-1].split('%')[0]) for k in pkeys])]
    
    for lead_time in np.arange(interval, lead_time_end, 6):
        
        for esize in ['Small', 'Medium', 'Large']:
            
            thresh = [thresholds[interval][thresh_id[esize][0]], 
                      thresholds[interval][thresh_id[esize][1]]]
        
            print('\rProcessing interval %d lead %dh'%(interval, lead_time), end='')

            # We need to break out the verification to each lead time,
            # but within each lead time we have a number of valid times.
            # At each lead time, valid time, isolate the forecast verification

            # Combine the datasets to make it easier to work with
            idata = nbm[nbm['LeadTime'] == lead_time].merge(obs, on='ValidTime').drop(columns='LeadTime')

            # Subset for event size
            iobs = idata['%dh_precip_in'%interval]
            idata = idata[((iobs >= thresh[0]) & (iobs < thresh[1]))]

            for itime in idata.index:

                try:
                    prob_fx = idata.loc[itime, pkeys].values
                    mean_fx = np.nanmean(prob_fx)
                    std_fx = np.nanstd(prob_fx)
                    med_fx = idata.loc[itime, 'APCP%dhr_surface_50%% level'%interval]
                    det_fx = idata.loc[itime, 'APCP%dhr_surface'%interval]

                    # Optional - leave as nan?
                    det_fx = det_fx if ~np.isnan(det_fx) else 0.

                    verif_ob = idata.loc[itime, '%dh_precip_in'%interval]
                    
                    verif_rank = np.searchsorted(prob_fx, verif_ob, 'right')                    
                    verif_rank_val = prob_fx[verif_rank-1]
                    verif_rank_error = verif_rank_val - verif_ob
                    
                    verif_rank = 101 if ((verif_rank >= 99) & (verif_ob > verif_rank_val)) else verif_rank
                    verif_rank = -1 if ((verif_rank <= 1) & (verif_ob < verif_rank_val)) else verif_rank
                    
                    det_rank = np.searchsorted(prob_fx, det_fx, 'right')
                    det_error = det_fx - verif_ob

                except:
                    raise
                    # pass
                    # print('failed', itime)

                else:
                    if ((verif_ob > 0.) & ~np.isnan(verif_rank_val)):

                        data.append([
                            # Indexers
                            interval, lead_time, itime, esize,

                            # Verification and deterministic
                            verif_ob, det_fx, det_rank, det_error,

                            # Probabilistic
                            verif_rank, verif_rank_val, verif_rank_error, 
                            med_fx, mean_fx, std_fx])

data = pd.DataFrame(data, columns=['Interval', 'LeadTime', 'ValidTime', 'EventSize',
                'verif_ob', 'det_fx', 'det_rank', 'det_error',
                'verif_rank', 'verif_rank_val', 'verif_rank_error', 
                'med_fx', 'mean_fx', 'std_fx'])

print('\n\nAvailable keys:\n\t\t{}\nn rows: {}'.format('\n\t\t'.join(data.keys()), len(data)))

***
***
# Create Bulk Temporal Stats Plots
#### Reliability diagrams, bias over time, rank over time, etc.

#### Plot histograms of percentile rank

In [ ]:
short, long = 0, 120
plot_type = 'Verification'
plot_var = 'verif_rank'
esize = 'All'

for interval in [6, 12, 24]:

    kwargs = {'_interval':interval, '_esize':esize,
             '_short':short, '_long':long,
             '_plot_type':plot_type, '_plot_var':plot_var}
    
    nbm_funcs.histograms_verif_rank(data, **kwargs)

#### Plot a reliability diagram style CDF to evaluate percentile rankings

In [ ]:
short, long = 0, 120
plot_type = 'Verification'
plot_var = 'verif_rank'
esize = 'All'

for interval in [6, 12, 24]:

    kwargs = {'_interval':interval, '_esize':esize,
             '_short':short, '_long':long,
             '_plot_type':plot_type, '_plot_var':plot_var}

    nbm_funcs.reliability_verif_cdf(data, **kwargs)

#### Produce bias, ME, MAE, and percentile rank plots as they evolve over time
This helps illustrate at what leads a dry/wet bias may exist and how severe it may be<br>
Adds value in interpreting the CDF reliability diagrams

In [ ]:
short, long = 0, 120
esize = 'All'

for interval in [6, 12, 24]:

    kwargs = {'_interval':interval, '_esize':esize,
             '_short':short, '_long':long}

    nbm_funcs.rank_over_leadtime(data, **kwargs)